In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count, when
from pyspark.sql.functions import round as rnd

In [2]:
conf = SparkConf()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
dfTweets = spark.read.csv("./tmp/spark_output/datacsv/tweets/tweets-tratado.csv", header=True,
                          inferSchema=True)
dfTweets.columns

['Tweets', 'len', 'ID', 'Date', 'Geo', 'Source', 'SA', 'Estado', 'Coordenadas']

In [49]:
dfIDD = spark.read.csv("./tmp/spark_output/datacsv/idd/idd-tratado.csv", header=True,
                       inferSchema=True)
dfIDD.columns

['IDD_Continuo',
 'IDD_Faixa',
 'Nota_Bruta_IDD',
 'Estado_Curso',
 'Instituicao_Ensino',
 'Concluintes_Inscritos',
 'Concluintes_Participantes',
 'Concluintes_Participantes_Nota_Enem',
 'Prop_Concluintes_Participantes_Nota_Enem_IDD',
 'Curso']

In [13]:
dfEnem = spark.read.csv("./tmp/spark_output/datacsv/enem/enem-tratado-sample.csv", header=True,
                        inferSchema=True)
dfEnem.columns

['Treineiro',
 'Estado',
 'Presenca_CN',
 'Presenca_CH',
 'Presenca_LC',
 'Presenca_MT',
 'Nota_CN',
 'Nota_CH',
 'Nota_LC',
 'Nota_MT',
 'Status_Redacao',
 'Nota_Redacao']

In [71]:
dfEnemMediaNotaEstado = dfEnem.filter((dfEnem['Status_Redacao'] == 1) & (dfEnem['Presenca_CN'] == 1) & (dfEnem['Presenca_CH'] == 1) & (dfEnem['Presenca_LC'] == 1) & (dfEnem['Presenca_MT'] == 1)).groupby("Estado").agg(
    rnd(avg(col("Nota_CN")), 2).alias("Nota Ciências Naturais"),
    rnd(avg(col("Nota_CH")), 2).alias("Nota Ciências Humanas"),
    rnd(avg(col("Nota_LC")), 2).alias("Nota Linguagens e Códigos"),
    rnd(avg(col("Nota_MT")), 2).alias("Nota Matemática"),
    rnd(avg(col("Nota_Redacao")), 2).alias("Nota Redação"),
    count(when(col("Treineiro") == 1, True)).alias("Treineiros"),
    count(when(col("Treineiro") == 0, True)).alias("Não Treineiros"))

In [64]:
dfIddMediaEstado = dfIDD.groupby(col("Estado_Curso").alias("Estado")).agg(
    rnd(avg(col("IDD_Continuo")), 2).alias("IDD Continuo"),
    rnd(avg(col("IDD_Faixa")), 2).alias("IDD Faixa"),
    rnd(avg(col("Concluintes_Inscritos")), 2).alias("Concluintes Inscritos"),
    rnd(avg(col("Concluintes_Participantes")), 2).alias("Concluintes Participantes"),
    rnd(avg(col("Concluintes_Participantes_Nota_Enem")), 2).alias("Concluintes Participantes Nota Enem"),
    rnd(avg(col("Prop_Concluintes_Participantes_Nota_Enem_IDD")), 2).alias(
        "Prop Concluintes Participantes Nota Enem IDD"))

In [61]:
dfTweetsSAEstado = dfTweets.groupby("Estado").agg(
    count(when(col("SA") > 0, True)).alias("Tweets Positivos"),
    count(when(col("SA") < 0, True)).alias("Tweets Negativos"),
    count(when(col("SA") == 0, True)).alias("Tweets Neutros"))

In [66]:
dfInnerJoin = dfEnemMediaNotaEstado.join(dfIddMediaEstado, on="Estado", how="inner").join(
    dfTweetsSAEstado, on="Estado", how="inner")

In [68]:
dfInnerJoin.show(vertical=True)


-RECORD 0---------------------------------------------------------
 Estado                                       | Ceará             
 Nota Ciências Naturais                       | 471.16            
 Nota Ciências Humanas                        | 500.96            
 Nota Linguagens e Códigos                    | 515.45            
 Nota Matemática                              | 521.48            
 Nota Redação                                 | 597.87            
 Treineiros                                   | 3645              
 Não Treineiros                               | 18124             
 IDD Continuo                                 | 3.33              
 IDD Faixa                                    | 3.22              
 Concluintes Inscritos                        | 72.93             
 Concluintes Participantes                    | 65.74             
 Concluintes Participantes Nota Enem          | 52.7              
 Prop Concluintes Participantes Nota Enem IDD | 0.83          